<a href="https://colab.research.google.com/github/justinjunge/Convergent-Wisdom-Project/blob/main/June24_Sacred_Texts_Heatmap_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to the Convergent Wisdom project! In this module, we will use natural language processing to build tools that can generate similarity scores among inputted verses from the Hindu Bhagavad Gita, Muslim Qur'an, and Christian Bible. Using these scores, we created heatmaps that visualize regions of similarity and dissimilarity across the texts. This methodology is especially robust as it allows us to see where the world's major religions converge and diverge in their wisdom.

This first code block initializes necessary libraries and imports the religious texts.

In [ ]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import pandas as pd
import numpy as np
import requests
import random
import copy
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from io import StringIO
from scipy import stats
from google.colab import files
from string import ascii_letters


github_csv_url_Gita = 'https://raw.githubusercontent.com/justinjunge/Convergent-Wisdom-Project/main/CSV_files/Gita.csv'

response1 = requests.get(github_csv_url_Gita)
if response1.status_code == 200:
  df_Gita = pd.read_csv(StringIO(response1.text))
else:
  print('Failed to fetch CSV File from Github')

github_csv_url_Bible = 'https://raw.githubusercontent.com/justinjunge/Convergent-Wisdom-Project/main/jj_ChristianBible.csv'
response2 = requests.get(github_csv_url_Bible)
if response2.status_code == 200:
  df_Bible = pd.read_csv(StringIO(response2.text))
else:
  print('Failed to fetch CSV File from Github')

github_csv_url_Quran = 'https://raw.githubusercontent.com/justinjunge/Convergent-Wisdom-Project/main/CSV_files/Quran.csv'
response3 = requests.get(github_csv_url_Quran)
if response3.status_code == 200:
  df_Quran = pd.read_csv(StringIO(response3.text))
else:
  print('Failed to fetch CSV File from Github')

df_Bible_by_chapter = df_Bible.groupby(['book', 'chapter'])['text'].apply(lambda x: ' '.join(x)).to_frame().reset_index()
df_Bible_by_chapter['text'][0]

df_Gita_by_verse = df_Gita.groupby(['Chapter', 'Verse'])['Text'].apply(lambda x: ' '.join(x)).to_frame().reset_index()
df_Gita_by_verse['Text'][0]

df_Quran_by_chapter = df_Quran.groupby(['Surah', 'Ayah'])['Text'].apply(lambda x: ' '.join(x)).to_frame().reset_index()
df_Quran_by_chapter['Text'][0]

'In the name of Allah, Most Gracious, Most Merciful.'

In [ ]:
df_Quran

This code below installs required packages.

In [ ]:
!pip install -U sentence-transformers
!pip install torch torchvision torchaudio
import torch
!pip install transformers
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
!pip install matplotlib-venn
from matplotlib_venn import venn2, venn2_circles, venn3, venn3_circles
!pip install plotly
!pip install kaleido

!pip install nltk
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 27.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

This line below iniializes our model.

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

To properly conduct this type of processing on a large scale, we created vector embeddings of the Gita, Qur'an, and Bible. These embeddings are contained in .pt files in the Github repository. This code below initializes the vector embeddings. Please note that you will need to ensure that your runtime is set to GPU for the code to work.

In [ ]:
Gita_url = 'https://raw.githubusercontent.com/justinjunge/Convergent-Wisdom-Project/main/Vector_Embeddings/Gita_verse_Embeddings.pt'
Bible_url = 'https://raw.githubusercontent.com/justinjunge/Convergent-Wisdom-Project/main/Vector_Embeddings/Bible_chapter_Embeddings.pt'
Quran_url = 'https://raw.githubusercontent.com/justinjunge/Convergent-Wisdom-Project/main/Vector_Embeddings/Quran_chapter_Embeddings.pt'


# Send a GET request to fetch the file content
responseG = requests.get(Gita_url)
responseB = requests.get(Bible_url)
responseQ = requests.get(Quran_url)

# Check if the request was successful
if responseG.status_code == 200:
    with open('Gita_verse_Embeddings.pt', 'wb') as f:
      f.write(responseG.content)
      embeddingG = torch.load('Gita_verse_Embeddings.pt')
      print("Model loaded successfully!")
else:
    print("Failed to fetch the file.")

if responseB.status_code == 200:
    with open('Bible_chapter_Embeddings.pt', 'wb') as f:
        f.write(responseB.content)
    embeddingB = torch.load('Bible_chapter_Embeddings.pt')
    print("Model loaded successfully!")
else:
    print("Failed to fetch the file.")

if responseQ.status_code == 200:
    with open('Quran_chapter_Embeddings.pt', 'wb') as f:
        f.write(responseQ.content)
    embeddingQ = torch.load('Quran_chapter_Embeddings.pt')
    print("Model loaded successfully!")
else:
    print("Failed to fetch the file.")

Model loaded successfully!


<ipython-input-15-28fccdaf1e07>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddingB = torch.load('Bible_chapter.pt')


UnpicklingError: invalid load key, '\x0d'.

Our heatmaps will work based on calculated similarity scores among all verses of our religious texts. The code below provides an example where similarity scores can be generated for any two inputted sentences. The variables in this test block are also used in later analyses.

In [ ]:
sentence1 = "William James started the modern field of Psychology as both an empirical and theoretical domain. "
sentence2 = "Skittles are a snack food that is unhealthy."

# encode sentences to get their embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Sentence 1:", sentence1)
print("Sentence 2:", sentence2)
print("Similarity score:", cosine_scores.item())

Next, we will prepare our dataframes for heatmap analysis.

In [ ]:
columnG = df_Gita["Text"]
columnB = df_Bible["text"]
columnQ = df_Quran["Text"]

quoted_listG = ['"' + str(x) + '"' for x in columnG]
quoted_listB = ['"' + str(x) + '"' for x in columnB]
quoted_listQ = ['"' + str(x) + '"' for x in columnQ]

We are ready now to finalize the data that will be used to plot our heatmap! This tool takes any input phrase and returns the top two matching verses from each religious text.

In [ ]:
# Returns top N [num_results] matches to a phrase from the Gita, Bible, & Quran

input_phrase = ["Love everyone and be kind to everyone."]
num_results = 2
embedding1 = model.encode(input_phrase, convert_to_tensor=True)


# compute similarity scores Gita, Bible, Quran
# Gita
cosine_scoresG = util.pytorch_cos_sim(embedding1, embeddingG)
score_listG = list(cosine_scoresG[0])
sorted_listG = sorted(score_listG)
top_NG = sorted_listG[-num_results:]
top_NG = sorted(top_NG, reverse=True)
# Bible
cosine_scoresB = util.pytorch_cos_sim(embedding1, embeddingB)
score_listB = list(cosine_scoresB[0])
sorted_listB = sorted(score_listB)
top_NB = sorted_listB[-num_results:]
top_NB = sorted(top_NB, reverse=True)
# Quran
cosine_scoresQ = util.pytorch_cos_sim(embedding1, embeddingQ)
score_listQ = list(cosine_scoresQ[0])
sorted_listQ = sorted(score_listQ)
top_NQ = sorted_listQ[-num_results:]
top_NQ = sorted(top_NQ, reverse=True)


print(input_phrase)

for i in range(len(top_NG)):
  which_oneG = score_listG.index(top_NG[i])
  print('Gita', top_NG[i], quoted_listG[which_oneG], df_Gita.iloc[which_oneG,0], df_Gita.iloc[which_oneG,1])

for i in range(len(top_NB)):
  which_oneB = score_listB.index(top_NB[i])
  print('Bible', top_NB[i], quoted_listB[which_oneB], df_Bible.iloc[which_oneB,0], df_Bible.iloc[which_oneB,1], df_Bible.iloc[which_oneB,2])

for i in range(len(top_NQ)):
  which_oneQ = score_listQ.index(top_NQ[i])
  print('Quran', top_NQ[i], quoted_listQ[which_oneQ], df_Quran.iloc[which_oneQ,0], df_Quran.iloc[which_oneQ,1], df_Quran.iloc[which_oneQ,2])


We are ready to plot our first heatmap! This code below generates a heatmap comparing the similarity/dissimilarity of every Gita verse to every Bible chapter.

In [ ]:
# compute similarity scores Bible vs. Bible
cosine_scoresBB = util.pytorch_cos_sim(embeddingB, embeddingB)
print(cosine_scoresBB.shape)
# rows: Gita, columns: Bible
scores_BB = cosine_scoresBB.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_BB, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Bible (Chapter)", y="Bible (Chapt)", color="Semantic Similarity"),)
fig.show()

In [ ]:
# compute similarity scores Gita vs. Gita
cosine_scoresGG = util.pytorch_cos_sim(embeddingG, embeddingG)
print(cosine_scoresGG.shape)
# rows: Gita, columns: Bible
scores_GG = cosine_scoresGG.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_GG, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Gita (Verse)", y="Gita (Verse)", color="Semantic Similarity"),)
fig.show()

In [ ]:
# compute similarity scores Quran vs. Quran
cosine_scoresQQ = util.pytorch_cos_sim(embeddingQ, embeddingQ)
print(cosine_scoresQQ.shape)
# rows: Quran, columns: Quran
scores_QQ = cosine_scoresQQ.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_QQ, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Quran (Chapter)", y="Quran (Chapter)", color="Semantic Similarity"),)
fig.show()

In [ ]:
# prompt: Update the following code to display text from the data frame df_Gita_by_verse, with the passage corresponding to the x entry in df_Gita_by_verse['Text'][x] and y entry of df_Gita_by_verse['Text'][y]: # compute similarity scores Gita vs. Gita
# cosine_scoresGG = util.pytorch_cos_sim(embeddingG, embeddingG)
# print(cosine_scoresGG.shape)
# # rows: Gita, columns: Bible
# scores_GG = cosine_scoresGG.detach(

# compute similarity scores Gita vs. Gita
cosine_scoresGG = util.pytorch_cos_sim(embeddingG, embeddingG)
print(cosine_scoresGG.shape)
# rows: Gita, columns: Bible
scores_GG = cosine_scoresGG.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_GG, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Gita (Verse)", y="Gita (Verse)", color="Semantic Similarity"),)

# Update the hover text to display the corresponding text from df_Gita_by_verse
fig.update_traces(hovertext=df_Gita_by_verse['Text'])

fig.show()


In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Assuming df_Gita_by_verse and embeddingG are already defined and properly loaded

# Compute similarity scores Gita vs. Gita
cosine_scoresGG = util.pytorch_cos_sim(embeddingG, embeddingG)
print(cosine_scoresGG.shape)

# Detach and convert to numpy
scores_GG = cosine_scoresGG.detach().cpu().numpy()

# Create hover text array from df_Gita_by_verse
hover_text = np.empty_like(scores_GG, dtype=object)
for i in range(0,10):
    for j in range(0,10):
        hover_text[i, j] = f"Gita Verse {i+1}: {df_Gita_by_verse['Text'][i]}<br>Gita Verse {j+1}: {df_Gita_by_verse['Text'][j]}"

# Show heatmap
fig = px.imshow(
    scores_GG,
    color_continuous_scale='RdYlBu_r',
    zmin=-.25,
    zmax=1,
    labels=dict(x="Gita (Verse)", y="Gita (Verse)", color="Semantic Similarity")
)

# Update the hover text to display the corresponding text from df_Gita_by_verse
fig.update_traces(hovertext=hover_text, hoverinfo="text")

fig.show()

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Assuming df_Gita_by_verse and embeddingG are already defined and properly loaded

# Compute similarity scores Gita vs. Gita
cosine_scoresGG = util.pytorch_cos_sim(embeddingG, embeddingG)
print(cosine_scoresGG.shape)

# Detach and convert to numpy
scores_GG = cosine_scoresGG.detach().cpu().numpy()

# Restrict to the first 10x10 part of the matrix for troubleshooting
scores_GG_sub = scores_GG[:10, :10]

# Create hover text array from df_Gita_by_verse for the first 10 lines
hover_text = np.empty_like(scores_GG_sub, dtype=object)
for i in range(10):
    for j in range(10):
        hover_text[i, j] = f"Gita Verse {i+1}: {df_Gita_by_verse['Text'].iloc[i]}<br>Gita Verse {j+1}: {df_Gita_by_verse['Text'].iloc[j]}"

# Show heatmap
fig = px.imshow(
    scores_GG_sub,
    color_continuous_scale='RdYlBu_r',
    zmin=-.25,
    zmax=1,
    labels=dict(x="Gita (Verse)", y="Gita (Verse)", color="Semantic Similarity")
)

# Update the hover text to display the corresponding text from df_Gita_by_verse
fig.update_traces(
    customdata=hover_text,
    hovertemplate="%{customdata}<extra></extra>"
)

fig.show()

In [ ]:
# Hover text display currently has upper limits, and 250x250 is near the limit, so this restricts the figure to the first 250 units and displays them

# Compute similarity scores Gita vs. Gita
cosine_scoresGG = util.pytorch_cos_sim(embeddingG, embeddingG)
print(cosine_scoresGG.shape)

# Detach and convert to numpy
scores_GG = cosine_scoresGG.detach().cpu().numpy()

# Restrict to the first 250x250 part of the matrix for troubleshooting
scores_GG_sub = scores_GG[:250, :250]

# Create hover text array from df_Gita_by_verse for the first 10 lines
hover_text = np.empty_like(scores_GG_sub, dtype=object)
for i in range(250):
    for j in range(250):
        hover_text[i, j] = f"Gita Verse {i+1}: {df_Gita_by_verse['Text'].iloc[i]}<br>Gita Verse {j+1}: {df_Gita_by_verse['Text'].iloc[j]}"

# Show heatmap
fig = px.imshow(
    scores_GG_sub,
    color_continuous_scale='RdYlBu_r',
    zmin=-.25,
    zmax=1,
    labels=dict(x="Gita (Verse)", y="Gita (Verse)", color="Semantic Similarity")
)

# Update the hover text to display the corresponding text from df_Gita_by_verse
fig.update_traces(
    customdata=hover_text,
    hovertemplate="%{customdata}<extra></extra>"
)

fig.show()

In [ ]:
# Hover text display currently has upper limits, and 100x100 is near the limit, so this restricts the figure to the first 100 units and displays them

# Compute similarity scores Bible verses Bible
cosine_scoresBB = util.pytorch_cos_sim(embeddingB, embeddingB)
print(cosine_scoresBB.shape)

# Detach and convert to numpy
scores_BB = cosine_scoresBB.detach().cpu().numpy()

# Restrict to the first 250x250 part of the matrix for troubleshooting
scores_BB_sub = scores_BB[:10, :10]

# Create hover text array from df_Gita_by_verse for the first 10 lines
hover_text = np.empty_like(scores_BB_sub, dtype=object)
for i in range(10):
    for j in range(10):
        hover_text[i, j] = f"Bible Chapter {i+1}: {df_Bible_by_chapter['text'].iloc[i]}<br>Bible Chapter {j+1}: {df_Bible_by_chapter['text'].iloc[j]}"

# Show heatmap
fig = px.imshow(
    scores_BB_sub,
    color_continuous_scale='RdYlBu_r',
    zmin=-.25,
    zmax=1,
    labels=dict(x="Bible (Chapter)", y="Bible (Chapter)", color="Semantic Similarity")
)

# Update the hover text to display the corresponding text from df_Gita_by_verse
fig.update_traces(
    customdata=hover_text,
    hovertemplate="%{customdata}<extra></extra>"
)

fig.show()

In [ ]:
df_Bible_by_chapter['text']

In [ ]:
df_Gita_by_verse

In [ ]:
hover_text

In [ ]:
# compute similarity scores Gita vs. Bible
cosine_scoresGB = util.pytorch_cos_sim(embeddingG, embeddingB)
print(cosine_scoresGB.shape)
# rows: Gita, columns: Bible
scores_GB = cosine_scoresGB.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_GB, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Bible (Chapter)", y="Gita (Verse)", color="Semantic Similarity"),)
fig.show()

In general, the Gita and the Bible seem fairly dissimilar to each other. Chapters 400-650 of the Bible represent a range of medium similarity with most of the Gita, roughly corresponding to the books of Esther, Job, Psalms, and Proverbs.

Our next heatmap compares the similarity / dissimilarity of every Qur'an verse to every Bible chapter.

In [ ]:
# compute similarity scores Qur'an vs. Bible
cosine_scoresQB = util.pytorch_cos_sim(embeddingQ, embeddingB)
print(cosine_scoresQB.shape)
# rows: Qur'an, columns: Bible
scores_QB = cosine_scoresQB.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_QB, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Bible (Chapter)", y="Quran (Chapter)", color="Semantic Similarity"),)
fig.show()

Discussion of Heatmap 2

Our last heatmap compares the similarity / dissimilarity of every Qur'an verse to every Gita verse.

In [ ]:
# compute similarity scores Qur'an vs. Gita
cosine_scoresQG = util.pytorch_cos_sim(embeddingQ, embeddingG)
print(cosine_scoresQG.shape)
# rows: Qur'an, columns: Gita
scores_QG = cosine_scoresQG.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_QG, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Gita (Chapter)", y="Quran (Chapter)", color="Semantic Similarity"),)
fig.show()

In [ ]:
# prompt: Modify this code to add a hover over ability to see the source text # compute similarity scores Qur'an vs. Gita
# cosine_scoresQG = util.pytorch_cos_sim(embeddingQ, embeddingG)
# print(cosine_scoresQG.shape)
# # rows: Qur'an, columns: Gita
# scores_QG = cosine_scoresQG.detach().cpu().numpy()
# # Show heatmap
# fig = px.imshow(scores_QG, color_continuous_scale='RdBu_r', labels=dict(x="Gita (Chapter)", y="Qura

# compute similarity scores Qur'an vs. Gita
cosine_scoresQG = util.pytorch_cos_sim(embeddingQ, embeddingG)
print(cosine_scoresQG.shape)
# rows: Qur'an, columns: Gita
scores_QG = cosine_scoresQG.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_QG, color_continuous_scale='RdBu_r', labels=dict(x="Gita (Chapter)", y="Quran (Chapter)", color="Semantic Similarity"),)
fig.update_traces(hovertemplate=
    '<b>Gita Verse:</b> %{y}<br>' +
    '<b>Quran Verse:</b> %{x}<br>' +
    '<b>Similarity Score:</b> %{z:.2f}<extra></extra>')
fig.show()


In [ ]:
Writings_url = 'https://raw.githubusercontent.com/justinjunge/Convergent-Wisdom-Project/master/Demo_Embedding2.pt'



responseW = requests.get(Writings_url)

if responseW.status_code == 200:
    with open('Demo_Embedding2.pt', 'wb') as f:
        f.write(responseW.content)
    embeddingW = torch.load('Demo_Embedding2.pt')
    print("Model loaded successfully!")
else:
    print("Failed to fetch the file.")

In [ ]:
# compute similarity scores Writings vs. Bible
cosine_scoresBW = util.pytorch_cos_sim(embeddingW, embeddingB)
print(cosine_scoresBW.shape)
# rows: Qur'an, columns: Bible
scores_BW = cosine_scoresBW.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_BW, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Bible (Chapter)", y="Writings (Book)", color="Semantic Similarity"),)
fig.show()

In [ ]:
# compute similarity scores Writings vs. Writings
cosine_scoresWW = util.pytorch_cos_sim(embeddingW, embeddingW)
print(cosine_scoresWW.shape)
# rows: Gita, columns: Bible
scores_WW = cosine_scoresWW.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_WW, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Writings (chapter)", y="Writings (Chapter)", color="Semantic Similarity"),)
fig.show()

In [ ]:
# compute similarity scores Writings vs. Writings
cosine_scoresGW = util.pytorch_cos_sim(embeddingG, embeddingW)
print(cosine_scoresGW.shape)
# rows: Gita, columns: Bible
scores_GW = cosine_scoresGW.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_GW, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Gita (Verse)", y="Writings (Chapter)", color="Semantic Similarity"),)
fig.show()

In [ ]:
# compute similarity scores Writings vs. Quran
cosine_scoresQW = util.pytorch_cos_sim(embeddingQ, embeddingW)
print(cosine_scoresQW.shape)
# rows: Gita, columns: Bible
scores_QW = cosine_scoresQW.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_QW, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Writings (Chapter)", y="Quran (Chapter)", color="Semantic Similarity"),)
fig.show()

In [ ]:
df_Bible_by_chapter = df_Bible.groupby(['book', 'chapter'])['text'].apply(lambda x: ' '.join(x)).to_frame().reset_index()
df_Bible_by_chapter['text'][0]

df_Gita_by_verse = df_Gita.groupby(['Chapter', 'Verse'])['Text'].apply(lambda x: ' '.join(x)).to_frame().reset_index()
df_Gita_by_verse['Text'][0]

df_Quran_by_chapter = df_Quran.groupby(['Surah', 'Ayah'])['Text'].apply(lambda x: ' '.join(x)).to_frame().reset_index()
df_Quran_by_chapter['Text'][0]


In [ ]:
df_Quran_by_chapter

In [ ]:
# compute similarity scores Bible vs. Bible
cosine_scoresBB = util.pytorch_cos_sim(embeddingB, embeddingB)
print(cosine_scoresBB.shape)
# rows: Gita, columns: Bible
scores_BB = cosine_scoresBB.detach().cpu().numpy()

# Show heatmap
fig = px.imshow(scores_BB, color_continuous_scale='RdYlBu_r', zmin = -.25, zmax = 1, labels=dict(x="Bible (Chapter)", y="Bible (Chapt)", color="Semantic Similarity"),)
fig.show()


Discussion of Heatmap 4

# Remarks

The application of natural language processing, sentiment analysis, and word cloud generation methodologies to the study of emotional religious psychology yields profound insights and opens countless new doors for discovery. In just one short notebook, our tools read through thousands of pages of sacred religious scripture and generated emotional data about them in seconds. When these data are paired with data from the lived experience of different religious practitioners, it becomes possible to see how life experiences and religious texts can both influence each others' emotional canvas. It also becomes possible to see how the world's great religions converge and diverge in their wisdom on human psychology.

While this cutting-edge interdisciplinary work is truly groundbreaking, the sheer potential for future researchers to catapult these methods to even greater heights is all the more exciting. The authors of this project can't wait to see how the global community of human scholars further expands on our initial ideas.

Have you checked out our other modules yet? See the Word Clouds and Sentiment Analysis modules for even more exciting possibilities.